In [1]:
# 90% CI from Monte Carlo
import pandas as pd
import numpy as np
pd.set_option('display.float_format', '{:.2e}'.format)

# Data import
file_path = "/Users/elchulito/Library/CloudStorage/OneDrive-polymtlus/0 - A_Database and methodology_PhD/PlasticFADE.xlsx"
sheet_name = "Uncertainty"
data_CI = pd.read_excel(file_path, sheet_name=sheet_name, usecols="A:D", skiprows=1)
data_CI = data_CI.iloc[48:54] # Row index minus 3, change this range for other polymers
print(data_CI)

# Parameter estimates
a_i, delta_i, b_i, alpha_i, c_i, beta_i = data_CI.iloc[:, 2].values
# Standard deviations of parameters
a_i_std, delta_i_std, b_i_std, alpha_i_std, c_i_std, beta_i_std = data_CI.iloc[:, 3].values

             Process Parameter  Estimate Standard deviation
48  PS fragmentation       a_i  6.41e-03           6.55e-02
49               NaN   delta_i  2.96e-01           1.28e+00
50               NaN       b_i  2.45e-07           5.93e-06
51               NaN   alpha_i  2.94e+00           6.20e+00
52               NaN       c_i  1.99e+02           2.11e-06
53               NaN    beta_i  1.61e+00           1.21e+00


In [3]:
# Input parameters
data_input = pd.read_excel(file_path, sheet_name="Results", usecols="A:E", skiprows=13)
data_input = data_input[data_input.iloc[:, 0] == "PS"]  # Change for different polymer types
print(data_input)
data_input.columns = ['Polymer', 'Compartment', 's', 'I_j', 'P_j']

# --- Prepare output lists ---
results = []

# --- Loop through each row of input ---
for index, row in data_input.iterrows():
    s = row['s']
    I_j = row['I_j']
    P_j = row['P_j']

    # Function to convert std to log-space std for lognormal sampling
    def log_std(mean, std):
        return np.sqrt(np.log(1 + (std / mean)**2))

    # Monte Carlo with log-normal distribution
    N = 10000
    a_i_samples = np.random.lognormal(np.log(a_i), log_std(a_i, a_i_std), N)
    delta_i_samples = np.random.lognormal(np.log(delta_i), log_std(delta_i, delta_i_std), N)
    b_i_samples = np.random.lognormal(np.log(b_i), log_std(b_i, b_i_std), N)
    alpha_i_samples = np.random.lognormal(np.log(alpha_i), log_std(alpha_i, alpha_i_std), N)
    c_i_samples = np.random.lognormal(np.log(c_i), log_std(c_i, c_i_std), N)
    beta_i_samples = np.random.lognormal(np.log(beta_i), log_std(beta_i, beta_i_std), N)

    # Compute k_frag for each sample
    k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)
    k_samples = k_samples[np.isfinite(k_samples)]  # Filter invalid samples (good habit, especially when NaNs are found in the CIs)
    
    # 90% CI in log10-space
    if np.all(k_samples == 0):
        lower_bound = 0
        upper_bound = 0
    else:
        log_k = np.log10(k_samples[k_samples > 0])  # Exclude zeros (for the soil compartment)
        log_lower = np.percentile(log_k, 5)
        log_upper = np.percentile(log_k, 95)
        lower_bound = 10 ** log_lower
        upper_bound = 10 ** log_upper
    k_point = a_i * (s**delta_i) * (b_i * I_j**alpha_i + c_i * P_j**beta_i)
    
    results.append({'k_point': k_point, 'CI_lower': lower_bound, 'CI_upper': upper_bound})

# --- Display results ---
results_CI = pd.DataFrame(results)
print("\n", results_CI)
print(f"\n{N - len(k_samples)} out of {N} samples were invalid and removed.")

   Polymer (i) Compartment (j)  SA:V [cm-1]  I_j [W/m2]  P_j [mW]
6           PS             Air           40    1.00e+01  3.17e-03
7           PS            Soil           40    0.00e+00  0.00e+00
8           PS           Beach           40    1.25e+01  4.70e-02
9           PS   Water surface           40    1.00e+01  2.07e-02
10          PS    Water column           40    0.00e+00  2.07e-08
11          PS        Sediment           40    0.00e+00  0.00e+00

    k_point  CI_lower  CI_upper
0 3.70e-04  2.44e-07  2.62e+22
1 0.00e+00  0.00e+00  0.00e+00
2 2.80e-02  3.05e-05  2.19e+25
3 7.48e-03  2.93e-06  9.94e+20
4 1.70e-12  7.85e-36  1.74e-01
5 0.00e+00  0.00e+00  0.00e+00

3 out of 10000 samples were invalid and removed.


/var/folders/_j/jvdb8rz100z48jv89nnnmdlw0000gn/T/ipykernel_79318/2653668137.py:30: RuntimeWarning: overflow encountered in power
  k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)
/var/folders/_j/jvdb8rz100z48jv89nnnmdlw0000gn/T/ipykernel_79318/2653668137.py:30: RuntimeWarning: invalid value encountered in multiply
  k_samples = a_i_samples * (s**delta_i_samples) * (b_i_samples * I_j**alpha_i_samples + c_i_samples * P_j**beta_i_samples)


In [5]:
# Write confidence intervals back to Excel (without modifying)
import xlwings as xw

wb = xw.Book(file_path)  # file_path is your existing Excel file path
sheet = wb.sheets["Results"]

start_row = 21  # Change this index for other polymers
sheet.range(f'K{start_row}').options(index=False, header=False).value = results_CI['CI_lower'].values.reshape(-1, 1)
sheet.range(f'L{start_row}').options(index=False, header=False).value = results_CI['CI_upper'].values.reshape(-1, 1)

wb.save()
wb.close()